In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() # Load environment variables from a .env file if present

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F7B0BAB3A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F7B0BA8F40>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
ans=model.invoke([HumanMessage(content="Hii, My name is Yash")])
ans

AIMessage(content="Hi Yash, it's nice to meet you! 👋\n\nIs there anything I can help you with today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 15, 'total_tokens': 43, 'completion_time': 0.050909091, 'prompt_time': 0.00124491, 'queue_time': 0.25216151, 'total_time': 0.052154001}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--5c0a93b4-79eb-4462-9e90-ed4e17396dd3-0', usage_metadata={'input_tokens': 15, 'output_tokens': 28, 'total_tokens': 43})

In [5]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hii, My name is Yash"),
    AIMessage(content="Hello Yash! It's nice to meet you. 👋\n\nWhat can I do for you today?\n"),
    HumanMessage(content="can you tell my name?")
])

AIMessage(content='Of course! Your name is Yash. 😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 53, 'total_tokens': 77, 'completion_time': 0.043636364, 'prompt_time': 0.00218821, 'queue_time': 0.250659599, 'total_time': 0.045824574}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8336b758-596b-4ff0-91c1-f8f8f0e6eb46-0', usage_metadata={'input_tokens': 53, 'output_tokens': 24, 'total_tokens': 77})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Yash and I am a software developer.")],
    config=config
)

In [9]:
response.content

"Hi Yash! It's nice to meet you.  \n\nI'm glad you're here. What can I do for you today?  Are you working on any interesting projects?\n"

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Yash.  😊  \n\nI remembered from our first conversation!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 78, 'total_tokens': 97, 'completion_time': 0.034545455, 'prompt_time': 0.002359949, 'queue_time': 0.2519849, 'total_time': 0.036905404}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--23338e4e-db80-49ad-9cb3-edecb19b5251-0', usage_metadata={'input_tokens': 78, 'output_tokens': 19, 'total_tokens': 97})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and don't know your name. If you'd like to tell me, I'm happy to use it! 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. 👋\n\nWhat can I do for you today?\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  😄  \n\nI remembered from our last conversation. Is there anything else I can help you with?\n'

## Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt=ChatPromptTemplate.from_messages([
    ("system"," You are a helpful assistant that helps people find information."),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hii, My name is Yash")]})

AIMessage(content="Hello Yash! 👋\n\nIt's nice to meet you. What can I help you find information about today? 😊  \n\nJust ask away, and I'll do my best to assist you. \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 26, 'total_tokens': 72, 'completion_time': 0.083636364, 'prompt_time': 0.00140131, 'queue_time': 0.251745299, 'total_time': 0.085037674}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e10e885c-957b-4d26-882c-1e8fe6b066d3-0', usage_metadata={'input_tokens': 26, 'output_tokens': 46, 'total_tokens': 72})

In [19]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [33]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="My name is Yash")],
    config=config
)
response.content

"Hi Yash! It's nice to meet you. \n\nHow can I help you today?  \n"

In [34]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name?")],
    config=config
)
response.content

'Your name is Yash! 😊 \n\nIs there anything else I can help you with, Yash?  \n'

In [28]:
prompt=ChatPromptTemplate.from_messages([
    ("system"," You are a helpful assistant that helps people find information in {language}."),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [29]:
response=chain.invoke(
    {"messages":[HumanMessage(content="Hii, My name is Yash")],
     "language":"Hindi"}
)

response.content

'नमस्ते Yash! 👋 \n\nमुझे खुशी है कि आप मुझसे जुड़े हैं। आप मुझे क्या जानकारी ढूंढने में मदद कर सकती हूँ? \n\n😊\n\n'

In [30]:
chain.invoke(
    {"messages":[HumanMessage(content="What is Gen AI?")],"language":"English"}
)

AIMessage(content="Gen AI stands for **Generative Artificial Intelligence**.  \n\nIt's a type of artificial intelligence that focuses on creating new content, rather than just analyzing existing data. Think of it like this:\n\n* **Traditional AI:** Analyzes your existing photos to identify what's in them (cat, dog, tree).\n* **Gen AI:** Creates a brand new photo of a cat playing with a ball of yarn.\n\nGen AI can create a wide range of content, including:\n\n* **Text:** Stories, poems, articles, code, dialogue\n* **Images:** Photos, artwork, illustrations\n* **Audio:** Music, sound effects, voiceovers\n* **Video:** Short clips, animations\n\nSome popular examples of Gen AI include:\n\n* **ChatGPT:** A chatbot that can hold human-like conversations and generate creative text formats.\n* **DALL-E 2:** An AI that creates realistic images from text descriptions.\n* **Midjourney:** Another AI art generator that produces stunning visuals.\n\nGen AI is rapidly evolving and has the potential t

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [35]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [37]:
config={"configurable":{"session_id":"chat4"}}

response=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is Yash")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते Yash!  \n\nखुश हूँ आपसे मिलकर।  😊 \n\nक्या मैं आपकी कोई मदद कर सकता हूँ?  \n'

In [38]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Whats my name?")],"language":"Hindi"},
    config=config
)
response.content

'आपका नाम Yash है।  😄 \n\n\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages